In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


In [2]:
weather = pd.read_csv("weather_data.csv")
weather.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Cape Town,-33.93,18.42,19.44,82,20,4.10,ZA,1604543581
1,1,Dikson,73.51,80.55,-16.69,91,0,2.48,RU,1604543554
2,2,Arrondissement de Saint-Malo,48.50,-1.83,6.00,87,0,3.10,FR,1604543663
3,3,Piteå,65.32,21.48,5.00,75,99,5.10,SE,1604543663
4,4,Muros,42.78,-9.06,11.67,66,90,2.24,ES,1604543634


In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather[["Lat", "Lng"]].astype(float)
humidity = weather["Humidity"]

In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
narrow_df = weather.loc[(weather["Max Temp"] < 35) & (weather["Max Temp"] > 25) \
                                    & (weather["Wind Speed"] < 10) \
                                    & (weather["Cloudiness"] == 0)]
narrow_df.dropna()
narrow_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,Cairns,-16.92,145.77,31.11,58,0,4.60,AU,1604543447
36,36,Atuona,-9.80,-139.03,26.03,77,0,9.30,PF,1604543581
62,62,Geraldton,-28.77,114.60,32.00,24,0,4.60,AU,1604543676
258,258,Srīvardhan,18.03,73.02,25.86,57,0,1.61,IN,1604543723
299,299,Trincomalee,8.57,81.23,27.93,75,0,2.24,LK,1604543734
306,306,Kulhudhuffushi,6.62,73.07,27.75,65,0,0.66,MV,1604543735
420,420,Tombouctou,20.00,-3.00,25.68,18,0,1.32,ML,1604543777
474,474,Bowen,-20.02,148.23,33.00,36,0,3.10,AU,1604543800
481,481,Edd,13.93,41.69,25.94,55,0,2.93,ER,1604543803
545,545,Māngrol,21.12,70.12,26.01,46,0,2.95,IN,1604543823


In [13]:
hotel_df = narrow_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
22,Cairns,AU,-16.92,145.77,
36,Atuona,PF,-9.80,-139.03,
62,Geraldton,AU,-28.77,114.60,
258,Srīvardhan,IN,18.03,73.02,
299,Trincomalee,LK,8.57,81.23,
306,Kulhudhuffushi,MV,6.62,73.07,
420,Tombouctou,ML,20.00,-3.00,
474,Bowen,AU,-20.02,148.23,
481,Edd,ER,13.93,41.69,
545,Māngrol,IN,21.12,70.12,


In [14]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params).json()

    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
22,Cairns,AU,-16.92,145.77,Mantra Trilogy Cairns
36,Atuona,PF,-9.80,-139.03,Villa Enata
62,Geraldton,AU,-28.77,114.60,Broadwater Mariner Resort
258,Srīvardhan,IN,18.03,73.02,Savli Resort/Hotel - Shrivardhan
299,Trincomalee,LK,8.57,81.23,Welcombe Hotel
306,Kulhudhuffushi,MV,6.62,73.07,Haajy Guest House 1
420,Tombouctou,ML,20.00,-3.00,
474,Bowen,AU,-20.02,148.23,Castle Motor Lodge Motel
481,Edd,ER,13.93,41.69,
545,Māngrol,IN,21.12,70.12,Hotel The Royal Honours


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
 # Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))